In [1]:
# Faiss
# Facebook AI similarity Search (FAISS) is a library for efficient
# similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any 
# size, upto ones that possibly don't fit in RAM. It also contains
# supporting code for evaluation and parameter tuning.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [5]:
loader = TextLoader('speech.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
docs = text_splitter.split_documents(documents)

Created a chunk of size 470, which is longer than the specified 200
Created a chunk of size 347, which is longer than the specified 200
Created a chunk of size 668, which is longer than the specified 200
Created a chunk of size 982, which is longer than the specified 200
Created a chunk of size 789, which is longer than the specified 200


In [8]:
embeddings = OllamaEmbeddings(model='gemma:2b')
db = FAISS.from_documents(docs, embeddings)
db

In [9]:
query = "what does the speaker describe the desired outcome of the war?"
docs = db.similarity_search(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [11]:
docs[0].page_content

'Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'

In [12]:
# As a Retriever
# We can also convert the vectorstore into retriever class.
# This allows us to easily use it in other Langchain method, which largely work with retrievers

In [14]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

In [15]:
docs[0]

Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.')

In [ ]:
# Similarity search with score 

# There are some FAISS specific methods. One of them is similarity search with score, which allows you to return
# not only the documents but also the distance score of the query to them. 
# The returned distance score is L2 distance. Therefore, a lower Score is better.

In [16]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
  2724.8784),
 (Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
  2853.0935),
 (Document(metadata={'sourc

In [17]:
embeddings_vector = embeddings.embed_query(query)
embeddings_vector

[0.18906155228614807,
 -2.2281577587127686,
 0.2358950823545456,
 1.1244100332260132,
 0.8516511917114258,
 0.4926098585128784,
 -0.49862927198410034,
 0.051157426089048386,
 0.17182417213916779,
 -0.9210087060928345,
 1.1472710371017456,
 -0.00031407797359861434,
 -1.2061717510223389,
 1.1102216243743896,
 -0.19198213517665863,
 -0.8631728887557983,
 3.052039384841919,
 1.4410260915756226,
 0.8437463045120239,
 0.5080544352531433,
 0.35531121492385864,
 -0.8756158947944641,
 -0.04407351836562157,
 1.4221196174621582,
 0.024296583607792854,
 -0.38284558057785034,
 -1.2826895713806152,
 -1.2013638019561768,
 -0.7018340826034546,
 -1.865691065788269,
 -0.011654377914965153,
 -1.3771027326583862,
 1.1201571226119995,
 -0.8392598032951355,
 -0.3415857255458832,
 -0.14194919168949127,
 1.784757375717163,
 0.4247511625289917,
 -0.028874199837446213,
 -0.32676753401756287,
 0.278756320476532,
 1.0043891668319702,
 0.7963266968727112,
 -1.1976217031478882,
 -1.4644280672073364,
 0.607976913452

In [18]:
db.similarity_search_by_vector(embeddings_vector)

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content

In [19]:
db.save_local("faiss_local")

In [22]:
new_df = FAISS.load_local("faiss_local", embeddings, allow_dangerous_deserialization=True)



In [23]:
new_df.similarity_search(query)

[Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.'),
 Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content